# MM-Fit Pose ↔ Accelerometer Alignment QA

Sample random windows from the aligned pose and standardized accelerometer files to ensure they cover the same frames.

In [1]:
from pathlib import Path
import numpy as np
import random
import matplotlib.pyplot as plt
import yaml

DATA_DIR = (Path.cwd() / '../datasets/mmfit').resolve()
CONFIG_PATH = (Path.cwd() / '../conf/data/mmfit.yaml').resolve()
print('Data dir:', DATA_DIR)
assert DATA_DIR.exists(), 'Missing mmfit dataset directory'
with CONFIG_PATH.open('r') as f:
    cfg = yaml.safe_load(f) or {}
POSE_FILE = cfg.get('pose_file', 'pose_3d_aligned.npy')
ACC_FILE = cfg.get('acc_file', 'sw_l_acc_std.npy')
print('Using pose file:', POSE_FILE)
print('Using acc file:', ACC_FILE)

Data dir: /Users/parham/Desktop/The Semester/code/datasets/mmfit
Using pose file: pose_3d_aligned.npy
Using acc file: sw_l_acc_std.npy


In [2]:
def load_pair(subject_dir: Path):
    pose = np.load(subject_dir / f"{subject_dir.name}_{POSE_FILE}")
    acc = np.load(subject_dir / f"{subject_dir.name}_{ACC_FILE}")
    return pose, acc

In [3]:
# load a single pair for demonstration
window_size = 400
random.seed(0)
subjects = sorted([d for d in DATA_DIR.iterdir() if d.is_dir()])
subject_dir = random.choice(subjects)
pose, acc = load_pair(subject_dir)
pose.shape, acc.shape

((3, 233290, 18), (233290, 5))

In [ ]:
window_size = 400
random.seed(0)
subjects = sorted([d for d in DATA_DIR.iterdir() if d.is_dir()])
num_subjects = min(3, len(subjects))
for subject_dir in random.sample(subjects, k=num_subjects):
    pose, acc = load_pair(subject_dir)
    max_start = pose.shape[1] - window_size
    start = random.randint(0, max_start)
    end = start + window_size
    pose_window = pose[:, start:end, 2:]
    acc_window = acc[start:end, 2:5].T
    fig, axs = plt.subplots(3, 3, figsize=(15, 8))
    fig.suptitle(f"{subject_dir.name} window {start}:{end}")
    axes_labels = ['x', 'y', 'z']
    for axis in range(3):
        axs[axis, 0].plot(acc_window[axis], color='tab:blue', label='Acc')
        axs[axis, 0].set_ylabel(f'Acc {axes_labels[axis]}')
        axs[axis, 0].grid(True, alpha=0.3)
        axs[axis, 0].legend(loc='upper right')
        axs[axis, 1].plot(pose_window[axis, :, 0], color='tab:orange', label='Pose j0')
        axs[axis, 1].set_ylabel(f'Pose j0 {axes_labels[axis]}')
        axs[axis, 1].grid(True, alpha=0.3)
        axs[axis, 1].legend(loc='upper right')
        joint_idx = min(pose_window.shape[2] - 1, 2)
        axs[axis, 2].plot(pose_window[axis, :, joint_idx], color='tab:green', label=f'Pose j{joint_idx}')
        axs[axis, 2].set_ylabel(f'Pose j{joint_idx} {axes_labels[axis]}')
        axs[axis, 2].grid(True, alpha=0.3)
        axs[axis, 2].legend(loc='upper right')
    axs[-1, 0].set_xlabel('timestep')
    axs[-1, 1].set_xlabel('timestep')
    axs[-1, 2].set_xlabel('timestep')
    plt.tight_layout(rect=[0, 0, 1, 0.94])
    plt.show()